# EA.01: Modellierung von GA
1. Färbeproblem mit 5 verschiedenen Farben, 
    Ziel: Konfliktfreie Färbung mit einer minimalen Anzahl an Farben. 

2. 8-Queens-Problem 

Geben Sie an:  
- Geeignete Kodierung der Individuen 
- passende Operatoren wie Crossover oder Mutation 
- Fitnessfunktion

-> Begründen Sie Ihre Wahl!

Was würden Sie noch benötigen um die Probleme mit Simulated Annealing lösen zu können? 

## Färbeproblem 

1. Codierung der Individuen mit einem Array, das die Farben angibt: [A, B, C, D, E, F] (R, G, B, Y, M)

2. Es soll einen Crossover zwischen den Eltern geben mit einer Wahrscheinlichkeit von 60%. Trennung an zufälliger Stelle
    - Außerdem soll es Mutation geben, bei dem die Mutationswahrscheinlichkeit bei jedem Gen eines Individuum 0.01 beträgt. 
    - Alternativ könnte die mutationsrate auch 1/6 betragen

3. Die Fitnessfunktion hat eine Sekundäre und Primäre Bedingung.
    - Primäre Bedingung: Minimale Anzahl an Farben 
    - Sekundäre Bedingung: Konfliktfreie Färbung 
    - (-10\*n_f) + (-2\*n_k)
    - n_f = Anzahl der insgesamten Farben 
    - n_k = Anzahl der Konflikte bei der Färbung
    - max. fitness = -15


## 8-Queens-Problem
_Kann ich die Anzahl der Damen so Kontrollieren oder soll ich das durch die Fitnessfunktion geben?_


1. Codierung der Individuen durch ein Spielfeld in dem die Königinnen schon gesetzt sind. Da nur eine Königin pro Reihe existieren kann wird auch nur eine Königin pro Reihe gesetzt. Die 1-8 geben die Stelle in der Reihe an. [1, 2, 3, 4, 5, 6, 7, 8] (1-8) 

2. Crossover mit 60% Wahrscheinlichkeit. Dabei soll die Trennung an einer zufälligen Stelle geschehen. 
    - Mutation mit einer Wahrscheinlichkeit von 0.01 oder 1/8

3. Die Fitnessfunktion 
    - -1, wenn eine Königin in der Nähe 
    - -1, wenn gleiche Zahl vorhanden, da diese beiden sich dann treffen können.
    - (-1\*n_n) + (-1\*n_r)
    - n_n = Anzahl der Königinnen in der Nähe
    - n_r = Anzahl der Königinnen in der gleichen Reihe 
    - max. fitness = 0 


## Was würden Sie noch benötigen, um die obigen Probleme jeweils mit Simulated Annealing lösen zu können?
- Es müsste noch festgelegt werden wie die fittesten Individuuen ausgewählt werden. 
- Dabei könnte man die Roulette-Selektion oder die Turnier-Selektion nehmen 


In [181]:
import random as r


class Faerbeproblem: 

    colors = ["R", "G", "B", "Y", "M"]

    def __init__(self, max_gen: int = 100, n_pop: int = 100, n_tournaments = 10, tournament_size = 10, recombination_probability = 60, mutation_probability = 1):
        self.max_gen = max_gen
        self.n_pop = n_pop
        self.n_tournaments = n_tournaments
        self.tournament_size = tournament_size
        self.recombination_probability = recombination_probability
        self.mutation_probability = mutation_probability

    def init_population(self):
        self.population = [[r.choice(self.colors) for i in range(6)] for i in range(self.n_pop)]

    def fitness(self, individual: list[str]): 
        """Determines the fitness for one individual"""
        n_f = len(set(individual))
        #Constraints: A/=B,  A/=C, B/=C, B/=D, C/=D, D/=E
        constraints = {(0,1), (0,2), (1,2), (1,3), (2,3), (3,4)}

        n_k = 0
        for i in constraints:
            if individual[i[0]] == individual[i[1]]:
                n_k+=1
        
        #Maximale Fitness ist -15
        return -15 / (n_k*-3 + n_f*-5)

    def recombination(self, parent1: list[str], parent2: list[str]):
        if len(parent1) != len(parent2): 
            raise
        if r.randint(1,100) > self.recombination_probability: 
            return parent1, parent2
        
        child1 = parent1.copy()
        child2 = parent2.copy()
        #zufällig trennen 
        cut = r.randint(1, len(parent1)-1)
        #Gene tauschen 
        for i in range(cut):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
    
        return child1, child2
        
    def mutation(self, individual: list[str]):
        #Chance von 0.01
        mutated = individual.copy()
        for i in range(len(individual)): 
            if r.randint(1,100) > self.mutation_probability: 
                mutated[i] = r.choice(self.colors)
        return mutated 

    def selection(self):
        mating_pool = []
        tournament_pool = []

        for i in range(self.n_tournaments): 
            for j in range(self.tournament_size):
                tournament_pool.append(r.choice(self.population))

            fittest_individual = tournament_pool[0]

            for j in tournament_pool:
                if self.fitness(j) > self.fitness(fittest_individual):
                    fittest_individual = j
            tournament_pool = []
            mating_pool.append(fittest_individual)
        
        return mating_pool

    def let_the_games_begin(self): 
        self.init_population()
        curr_gen = 0
        new_population = []
        while self.max_gen > curr_gen: 
            print(self.population)
            population_fitness = [self.fitness(i) for i in self.population]
            
            # 2. Abbruchbedingung 
            if 1.0 in population_fitness: 
                print("Lösung gefunden:" + str(self.population[population_fitness.index(1.0)]))
                print("nach: "+ str(curr_gen) +" Generationen")
                break
            # Selection 
            mating_pool = self.selection()

            # Recombination
            for i in range(int(self.n_pop/2)):
                x,y = self.recombination(r.choice(mating_pool), r.choice(mating_pool))
                new_population.append(x)
                new_population.append(y)
                                
            # Mutation
            new_population = [self.mutation(i) for i in new_population]
            
            self.population = new_population.copy()
            new_population = []
            curr_gen+=1
f = Faerbeproblem(n_pop=10, tournament_size=5, n_tournaments=5)
f.fitness(['B', 'B', 'M', 'G', 'B', 'G'])


0.8333333333333334

In [192]:

f.let_the_games_begin()

[['B', 'G', 'G', 'B', 'M', 'G'], ['Y', 'M', 'B', 'R', 'G', 'Y'], ['B', 'Y', 'B', 'G', 'G', 'Y'], ['Y', 'B', 'M', 'R', 'B', 'M'], ['G', 'Y', 'R', 'G', 'Y', 'B'], ['Y', 'G', 'G', 'R', 'Y', 'Y'], ['Y', 'G', 'B', 'M', 'R', 'B'], ['R', 'B', 'G', 'B', 'G', 'M'], ['R', 'Y', 'R', 'B', 'R', 'B'], ['M', 'R', 'B', 'R', 'B', 'B']]
[['Y', 'G', 'B', 'Y', 'Y', 'G'], ['Y', 'R', 'Y', 'Y', 'Y', 'B'], ['B', 'Y', 'R', 'R', 'M', 'B'], ['M', 'R', 'R', 'M', 'G', 'R'], ['Y', 'G', 'M', 'M', 'R', 'R'], ['R', 'G', 'M', 'G', 'R', 'G'], ['R', 'G', 'B', 'G', 'B', 'Y'], ['Y', 'B', 'Y', 'R', 'M', 'B'], ['G', 'R', 'B', 'B', 'B', 'G'], ['B', 'G', 'M', 'Y', 'G', 'G']]
[['G', 'R', 'Y', 'B', 'M', 'G'], ['M', 'G', 'Y', 'M', 'B', 'R'], ['B', 'M', 'G', 'B', 'R', 'B'], ['Y', 'R', 'R', 'Y', 'Y', 'M'], ['G', 'R', 'M', 'M', 'R', 'R'], ['G', 'M', 'B', 'B', 'M', 'M'], ['M', 'Y', 'R', 'R', 'B', 'G'], ['Y', 'R', 'G', 'Y', 'R', 'G'], ['M', 'R', 'M', 'B', 'M', 'B'], ['R', 'Y', 'R', 'R', 'M', 'Y']]
Lösung gefunden:['Y', 'R', 'G', 'Y', 

1.0